#### Admissions (fact_admissions)

🎯 Gain

1 row = 1 encounter

📊 Use cases

Admissions count

Length of stay (LOS)

Admission type analysis

Operational dashboards

In [0]:
%sql
CREATE OR REPLACE TABLE angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
USING DELTA
AS
SELECT
    e.encounter_id,
    e.patient_id,

    e.admit_time,
    e.discharge_time,

    -- Derived metric
    DATEDIFF(e.discharge_time, e.admit_time) AS length_of_stay_days,

    e.status           AS encounter_status,
    e.admission_type,

    p.gender,
    p.birth_date,

    YEAR(e.admit_time)  AS admit_year,
    MONTH(e.admit_time) AS admit_month

FROM angad_kumar91.fhir_healthcare_analytics_silver.encounter e
JOIN angad_kumar91.fhir_healthcare_analytics_silver.patient p
  ON e.patient_id = p.patient_id;


#### GOLD FACT 2 — Labs / Observations (fact_labs)

🎯 Gain

1 row = 1 observation

📊 Use cases

Lab trends

Vitals monitoring

Clinical dashboards

Risk modeling inputs

In [0]:
%sql
CREATE OR REPLACE TABLE angad_kumar91.fhir_healthcare_analytics_gold.fact_labs
USING DELTA
AS
SELECT
    o.observation_id,
    o.patient_id,
    o.encounter_id,

    o.observation_name,
    o.value,
    o.unit,
    o.observation_time,

    e.admission_type,
    e.status AS encounter_status,

    YEAR(o.observation_time)  AS observation_year,
    MONTH(o.observation_time) AS observation_month

FROM angad_kumar91.fhir_healthcare_analytics_silver.observation o
LEFT JOIN angad_kumar91.fhir_healthcare_analytics_silver.encounter e
  ON o.encounter_id = e.encounter_id;


#### GOLD FACT 3 — Costs / Claims (fact_costs)

🎯 Gain

1 row = 1 claim (EOB)

📊 Use cases

Cost analysis

Insurance reporting

Revenue dashboards

Financial KPIs

In [0]:
%sql
CREATE OR REPLACE TABLE angad_kumar91.fhir_healthcare_analytics_gold.fact_costs
USING DELTA
AS
SELECT
    eob.eob_id,
    eob.patient_id,

    eob.claim_type,
    eob.claim_use,
    eob.insurer,

    eob.total_amount,
    eob.claim_status,

    eob.created_date,

    YEAR(eob.created_date)  AS claim_year,
    MONTH(eob.created_date) AS claim_month

FROM angad_kumar91.fhir_healthcare_analytics_silver.explanation_of_benefit eob;


_In the Gold layer, we designed separate fact tables for admissions, labs, and costs, each with a clearly defined grain. This avoids metric inflation, improves query performance, and aligns with dimensional modeling best practices_

**Validation queries**

In [0]:
%sql
SELECT COUNT(*) FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions;


In [0]:
%sql
SELECT admission_type, COUNT(*) 
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
GROUP BY admission_type
ORDER BY COUNT(*) DESC;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT observation_name, AVG(value)
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_labs
GROUP BY observation_name;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT insurer, SUM(total_amount) AS total_cost
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_costs
GROUP BY insurer;


### ⭐ Patient dimension table

**GOLD DIMENSION — dim_patient**

🎯 Purpose

A single, trusted patient master used across all dashboards and facts.

Grain

1 row = 1 patient

✅ Final Patient Dimension Schema

In [0]:
%sql
-- Patient Dimension

CREATE OR REPLACE TABLE angad_kumar91.fhir_healthcare_analytics_gold.dim_patient
USING DELTA
AS
SELECT
    patient_id,

    gender,
    birth_date,

    -- Age calculation (safe, dynamic)
    FLOOR(DATEDIFF(CURRENT_DATE(), birth_date) / 365.25) AS age_years,

    -- Deceased flag
    CASE 
        WHEN deceased_datetime IS NOT NULL THEN TRUE 
        ELSE FALSE 
    END AS is_deceased,

    marital_status,
    city,
    state,
    country,
    postal_code,
    preferred_language,
    language_code,
    phone_number,

    source_file,
    ingest_time

FROM angad_kumar91.fhir_healthcare_analytics_silver.patient;


#### Validation Queries

_We created a Gold patient dimension table with one row per patient, including derived attributes like age and deceased status. This dimension is shared across all fact tables to ensure consistent patient-level analytics and optimal BI performance_

In [0]:
%sql
SELECT COUNT(*) 
FROM angad_kumar91.fhir_healthcare_analytics_gold.dim_patient;


In [0]:
%sql
SELECT gender, COUNT(*) 
FROM angad_kumar91.fhir_healthcare_analytics_gold.dim_patient
GROUP BY gender;


In [0]:
%sql
SELECT is_deceased, COUNT(*) 
FROM angad_kumar91.fhir_healthcare_analytics_gold.dim_patient
GROUP BY is_deceased;


#### 🗓️ Time Dimension (dim_date) & 
#### 🩺 Diagnosis Dimension (dim_diagnosis)

#### what we are going to do now (BIG PICTURE)

We will build :

⭐ Dimensions

dim_patient

dim_date

dim_diagnosis

⭐ Facts

fact_admissions

fact_labs

fact_costs

👉 This is a complete healthcare analytics star schema.

GOLD DIMENSION 1 — Time Dimension (dim_date)
🎯 Purpose

Centralized calendar table for all time-based analysis:

Daily / Monthly / Yearly trends

YOY / MOM analysis

BI tool slicing (Power BI, Tableau)

Grain

1 row = 1 calendar date

In [0]:
%sql
CREATE OR REPLACE TABLE angad_kumar91.fhir_healthcare_analytics_gold.dim_date
USING DELTA
AS
WITH date_bounds AS (
    SELECT
        MIN(admit_time) AS min_date,
        MAX(discharge_time) AS max_date
    FROM angad_kumar91.fhir_healthcare_analytics_silver.encounter
),
date_series AS (
    SELECT explode(
        sequence(
            DATE(min_date),
            DATE(max_date),
            INTERVAL 1 DAY
        )
    ) AS date_key
    FROM date_bounds
)
SELECT
    date_key,

    YEAR(date_key)          AS year,
    QUARTER(date_key)       AS quarter,
    MONTH(date_key)         AS month,
    DATE_FORMAT(date_key, 'MMMM') AS month_name,

    DAY(date_key)           AS day_of_month,
    DAYOFWEEK(date_key)     AS day_of_week,
    DATE_FORMAT(date_key, 'EEEE') AS day_name,

    WEEKOFYEAR(date_key)    AS week_of_year,

    CASE
        WHEN DAYOFWEEK(date_key) IN (1, 7) THEN TRUE
        ELSE FALSE
    END AS is_weekend

FROM date_series;


In [0]:
%sql
SELECT COUNT(*) FROM angad_kumar91.fhir_healthcare_analytics_gold.dim_date;


In [0]:
%sql
SELECT year, COUNT(*) 
FROM angad_kumar91.fhir_healthcare_analytics_gold.dim_date
GROUP BY year;


### joining to FACT tables

Fact table using - 	Join column

fact_admissions  - 	DATE(admit_time)

fact_labs	  -   DATE(observation_time)

fact_costs	-    DATE(created_date)

In [0]:
%sql
SELECT
    d.year,
    COUNT(a.encounter_id) AS admissions
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions a
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_date d
  ON DATE(a.admit_time) = d.date_key
GROUP BY d.year;


Databricks visualization. Run in Databricks to view.

### 🩺 Diagnosis Dimension (dim_diagnosis)

### GOLD DIMENSION 2 — Diagnosis Dimension (dim_diagnosis)

🎯 Purpose

Central reference for all clinical diagnoses:

Disease prevalence

Risk analysis

Clinical dashboards

Grain

1 row = 1 unique diagnosis

✅ Final Schema — dim_diagnosis

🧠 Design Notes

Diagnoses come from silver.condition.diagnosis

We create a surrogate key

Category is optional but useful for analytics

In [0]:
%sql
--- Build Diagnosis Dimension
CREATE OR REPLACE TABLE angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis
USING DELTA
AS
SELECT
    monotonically_increasing_id() AS diagnosis_key,
    diagnosis                      AS diagnosis_name,

    -- Simple high-level categorization (extendable)
    CASE
        WHEN LOWER(diagnosis) LIKE '%diabetes%' THEN 'Metabolic'
        WHEN LOWER(diagnosis) LIKE '%hypertension%' THEN 'Cardiovascular'
        WHEN LOWER(diagnosis) LIKE '%infection%' THEN 'Infectious'
        WHEN LOWER(diagnosis) LIKE '%virus%' THEN 'Infectious'
        WHEN LOWER(diagnosis) LIKE '%renal%' THEN 'Renal'
        WHEN LOWER(diagnosis) LIKE '%depression%' THEN 'Mental Health'
        ELSE 'Other'
    END AS diagnosis_category

FROM (
    SELECT DISTINCT diagnosis
    FROM angad_kumar91.fhir_healthcare_analytics_silver.condition
    WHERE diagnosis IS NOT NULL
);


In [0]:
%sql
SELECT COUNT(*) 
FROM angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis;


In [0]:
%sql
SELECT diagnosis_category, COUNT(*) 
FROM angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis
GROUP BY diagnosis_category;


Databricks visualization. Run in Databricks to view.


_Till now We implemented a star schema in the Gold layer with dedicated fact tables for admissions, labs, and costs, supported by reusable dimensions such as patient, time, and diagnosis. This design ensures scalability, correctness of metrics, and optimal BI performance._

#### joining to FACT tables

For example, linking to admissions:

In [0]:
%sql
SELECT
    d.diagnosis_category,
    COUNT(*) AS cases
FROM angad_kumar91.fhir_healthcare_analytics_silver.condition c
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis d
  ON c.diagnosis = d.diagnosis_name
GROUP BY d.diagnosis_category;


### ⭐ Linking diagnosis to admissions fact &

### ⭐ Risk & utilization scoring fact

### 1️⃣ Linking Diagnosis to Admissions Fact(fact_admission_diagnosis)

🎯 we need this for:

One encounter can have multiple diagnoses

Directly joining diagnosis to fact_admissions would inflate metrics

Correct solution = bridge (factless fact) table

🧠 Grain

1 row = 1 encounter–diagnosis relationship

In [0]:
%sql
--- Build Admission–Diagnosis Bridge

CREATE OR REPLACE TABLE angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis
USING DELTA
AS
SELECT
    c.encounter_id,
    c.patient_id,
    d.diagnosis_key,
    d.diagnosis_name
FROM angad_kumar91.fhir_healthcare_analytics_silver.condition c
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis d
  ON c.diagnosis = d.diagnosis_name;


In [0]:
%sql
SELECT COUNT(*) 
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis;


In [0]:
%sql
SELECT diagnosis_name, COUNT(*) AS cases
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis
GROUP BY diagnosis_name
ORDER BY cases DESC;


In [0]:
%sql
SELECT
    d.diagnosis_category,
    COUNT(DISTINCT f.encounter_id) AS admissions
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis f
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis d
  ON f.diagnosis_key = d.diagnosis_key
GROUP BY d.diagnosis_category;


### 2️⃣ Risk & Utilization Scoring Fact(fact_patient_risk)

This is high-value analytics — recruiters LOVE this.

🎯 Goal

Create a patient-level scoring fact that measures:

Utilization (visits, claims)

Clinical burden (diagnoses)

Financial impact (costs)

🧠 Grain

1 row = 1 patient

In [0]:
%sql
--- Build Risk & Utilization Fact
CREATE OR REPLACE TABLE angad_kumar91.fhir_healthcare_analytics_gold.fact_patient_risk
USING DELTA
AS
WITH encounter_metrics AS (
    SELECT
        patient_id,
        COUNT(DISTINCT encounter_id) AS encounter_count
    FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
    GROUP BY patient_id
),
diagnosis_metrics AS (
    SELECT
        patient_id,
        COUNT(DISTINCT diagnosis_key) AS diagnosis_count
    FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis
    GROUP BY patient_id
),
cost_metrics AS (
    SELECT
        patient_id,
        SUM(total_amount) AS total_cost
    FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_costs
    GROUP BY patient_id
)
SELECT
    p.patient_id,

    COALESCE(e.encounter_count, 0) AS encounter_count,
    COALESCE(d.diagnosis_count, 0) AS diagnosis_count,
    COALESCE(c.total_cost, 0) AS total_cost,

    -- Utilization score (0–100)
    LEAST(COALESCE(e.encounter_count, 0) * 10, 100) AS utilization_score,

    -- Risk score (weighted)
    (
        COALESCE(e.encounter_count, 0) * 0.4 +
        COALESCE(d.diagnosis_count, 0) * 0.4 +
        COALESCE(c.total_cost, 0) / 1000 * 0.2
    ) AS risk_score

FROM angad_kumar91.fhir_healthcare_analytics_gold.dim_patient p
LEFT JOIN encounter_metrics e ON p.patient_id = e.patient_id
LEFT JOIN diagnosis_metrics d ON p.patient_id = d.patient_id
LEFT JOIN cost_metrics c ON p.patient_id = c.patient_id;


**Validation Queries**

_We implemented a bridge table to correctly model the many-to-many relationship between encounters and diagnoses. On top of that, we built a patient-level risk and utilization fact by aggregating encounters, diagnoses, and costs into explainable scoring metrics._

In [0]:
%sql
SELECT COUNT(*) 
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_patient_risk;


In [0]:
%sql
SELECT
    MIN(risk_score),
    MAX(risk_score),
    AVG(risk_score)
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_patient_risk;


In [0]:
%sql
SELECT *
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_patient_risk
ORDER BY risk_score DESC
LIMIT 10;


Databricks visualization. Run in Databricks to view.

**🏆 Final Architecture Status**

⭐ Dimensions

dim_patient

dim_date

dim_diagnosis

⭐ Facts

fact_admissions

fact_labs

fact_costs

fact_admission_diagnosis

fact_patient_risk